The below code will build your database. This code assumes you have installed 'magenta' via pip INSTEAD of anaconda.

Note to TA: any reference to 'ECE324' in a file path can be safely replaced with 'path/to/'. A recommended file organization structure has been provided but is not strictly nessecary. 

This file will need to be ran for both datasets, if recreating the same process as utilized in the companion write-up. An example for 'lmd_wan' has been given for reference. Further files and processing will assume you have both unless explicitly stated.

In [1]:
!python3 .local/lib/python3.8/site-packages/magenta/scripts/convert_dir_to_note_sequences.py   \
    --input_dir='/path/to/lmd_wan/train/original'   \
    --output_file='path/to/lmd_wan/train/ns_original.tfrecord'   \
    --recursive

!python3 .local/lib/python3.8/site-packages/magenta/scripts/convert_dir_to_note_sequences.py   \
    --input_dir='path/to/lmd_wan/test/original'   \
    --output_file='path/to/lmd_wan/test/ns_original.tfrecord'   \
    --recursive

!python3 .local/lib/python3.8/site-packages/magenta/scripts/convert_dir_to_note_sequences.py   \
    --input_dir='path/to/lmd_wan/eval/original'   \
    --output_file='path/to/lmd_wan/eval/ns_original.tfrecord'   \
    --recursive

python3: can't open file '.local/lib/python3.8/site-packages/magenta/scripts/convert_dir_to_note_sequences.py': [Errno 2] No such file or directory
python3: can't open file '.local/lib/python3.8/site-packages/magenta/scripts/convert_dir_to_note_sequences.py': [Errno 2] No such file or directory
python3: can't open file '.local/lib/python3.8/site-packages/magenta/scripts/convert_dir_to_note_sequences.py': [Errno 2] No such file or directory


Next, mask songs for infill task

In [ ]:
import generate_embeddings
folders = ['train', 'test', 'eval']
ns_input_path = ''
masked_ns_output_path = ''
for f in folders:
    if f == 'train': 
        for i in ['', '2', '3']:
            ns_input_path = '~/ECE324/lmd_wan/' + f + '/ns_original' + i + '.tfrecord'
            masked_ns_output_path = '~/ECE324/lmd_wan/' + f + '/ns_masked' + i + '.tfrecord'
            print(ns_input_path, masked_ns_output_path)
            generate_embeddings.mask_ns(ns_input_path, masked_ns_output_path)
    else: 
        ns_input_path = '~/ECE324/lmd_wan/' + f + '/ns_original.tfrecord'
        masked_ns_output_path = '~/ECE324/lmd_wan/' + f + '/ns_masked.tfrecord'
        print(ns_input_path, masked_ns_output_path)
        generate_embeddings.mask_ns(ns_input_path, masked_ns_output_path)

Next, encode training, testing, and evaluation

In [ ]:
#note: train might be too large and will kill the kernel after 5-6 hours
import generate_embeddings
folders = [ 'train', 'test', 'eval']
type = ['original','masked']
for f in folders:
    for t in type:
        if f == 'train': 
            for i in ['', '2', '3']:
                ns_input_path = '~/ECE324/lmd_wan/' + f + '/ns_' + t  + i + '.tfrecord'
                encoded_ns_output_path = '~/ECE324/lmd_wan/' + f + '/encoded_' + t  + i + '.tfrecord'
                print(ns_input_path, encoded_ns_output_path)
                generate_embeddings.encode_ns(ns_input_path, encoded_ns_output_path)
        else:
            ns_input_path = '~/ECE324/lmd_wan/' + f + '/ns_' + t + '.tfrecord'
            encoded_ns_output_path = '~/ECE324/lmd_wan/' + f + '/encoded_' + t + '.tfrecord'
            print(ns_input_path, encoded_ns_output_path)
            generate_embeddings.encode_ns(ns_input_path, encoded_ns_output_path)

Concatenate all training .tfrecords if needed (may be nessecary due to kernel issue processing full dataset at once)

In [ ]:
import tensorflow as tf
import os
def concatenate_tfrecord(input_files, output_file):
    with tf.io.TFRecordWriter(output_file) as writer:
        for input_file in input_files:
            dataset = tf.data.TFRecordDataset(input_file)
            for record in dataset:
                writer.write(record.numpy())

input_files = [os.path.expanduser('~/ECE324/lmd_wan/train/encoded_masked1.tfrecord'), 
               os.path.expanduser('~/ECE324/lmd_wan/train/encoded_masked2.tfrecord'), 
               os.path.expanduser('~/ECE324/lmd_wan/train/encoded_masked3.tfrecord')]
output_file = os.path.expanduser('~/ECE324/lmd_wan/train/encoded_masked.tfrecord')

concatenate_tfrecord(input_files, output_file)

Turn all encodings into paired train-test embeddings

In [ ]:
import generate_embeddings
type = ['original','masked']
for t in type:
    train_input_path = '~/ECE324/lmd_wan/train/encoded_' + t + '.tfrecord*'
    test_input_path = '~/ECE324/lmd_wan/test/encoded_' + t + '.tfrecord*'
    generate_embeddings.embed_encoding(train_input_path, 
                                    test_input_path,
                                   '~/ECE324/lmd_wan/'+ t + '_embeddings' )